# Get missing persons

Use pip environment : cf. instructions.md



In [1]:
import spacy
import psycopg2
import pandas as pd
from spacy.tokens import Span
from spacy import displacy
from time import strftime, gmtime

In [2]:
from itables import init_notebook_mode, show
import re
from importlib import reload
import os

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import postgresql_functions as pgf

In [ ]:
# reload(pgf)

In [6]:
### connect to the local database
conn = psycopg2.connect(host="localhost", port = 5432, database="espace_intellectuel", 
                        user="postgres", password=dbw)
#conn

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: fe_sendauth: no password supplied


In [7]:
#conn.close()

In [8]:
q1 = """
select pk_mathshistory, "name", url, dates, length(biography) as eff, coreferenced_txt, biography 
from mathshistory.mathshistory m 
where pk_mathshistory in (103, 117, 133, 159, 186);
"""

In [9]:
result = pgf.sql_explore(q1, conn)
# print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

In [10]:
textes = pd.DataFrame(result[0])
textes.columns = ['id', 'name', 'url', 'dates', 'length_bio', 'coref_texte', 'biog']

In [11]:
textes.head()

,id,name,url,dates,length_bio,coref_texte,biog
0,103,Christopher Clavius,https://mathshistory.st-andrews.ac.uk/Biograph...,1538-1612,10616,Christopher Clavius was born in a German regio...,Christopher Clavius was born in a German regio...
1,117,Michael Mästlin,https://mathshistory.st-andrews.ac.uk/Biograph...,1550-1631,10122,Michael Mästlin was born in Göppingen which wa...,Michael Mästlin was born in Göppingen which wa...
2,133,Giuseppe Biancani,https://mathshistory.st-andrews.ac.uk/Biograph...,1566-1624,10035,Giuseppe Biancani's name also appears in name ...,Giuseppe Biancani's name also appears in its L...
3,159,Wilhelm Schickard,https://mathshistory.st-andrews.ac.uk/Biograph...,1592-1635,10272,Wilhelm Schickard's name is sometimes written ...,Wilhelm Schickard's name is sometimes written ...
4,186,Johannes Hevelius,https://mathshistory.st-andrews.ac.uk/Biograph...,1611-1687,10856,The first problem that we have to address is t...,The first problem that we have to address is t...


In [12]:
### Choose one document
txt = textes.iloc[1].coref_texte  #[:1600]
print(txt)

Michael Mästlin was born in Göppingen which was a village about 50 km east of Tübingen. Mästlin father, Jakob Mästlin, and Mästlin mother, Dorothea Simon, were both devout Lutherans and Michael was brought up in that faith and remained strongly committed to father throughout Mästlin life. Mästlin was the middle child of the family, having an older sister and a younger brother. Mästlin attended the monastic school in Königsbronn then, after Mästlin studies there, entered Tübingen University in 1568. [3]:-
As was the case with many young scholars including Kepler, Mästlin most famous student, [Mästlin ] did Mästlin undergraduate studies at a preparatory school and came to the university to take Mästlin final exams and pick up Mästlin baccalaureate degree.At Tübingen University Mästlin studied mathematics and astronomy for a Master's degree under Philipp Apian who was Peter Apian's son. In 1570, while a student, Mästlin purchased a copy of Copernicus's De revolutionibus from the widow of 

## EntityLinker

* https://spacy.io/api/entitylinker


In [11]:
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("entityLinker", last=True)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
doc = nlp(txt)

In [13]:
all_linked_entities = doc._.linkedEntities
for sent in doc.sents:
    sent._.linkedEntities.pretty_print()

<EntityElement: https://www.wikidata.org/wiki/Q75797 Michael Maestlin          German astronomer and mathematician               >
<EntityElement: https://www.wikidata.org/wiki/Q532 village                   small clustered human settlement smaller than a town>
<EntityElement: https://www.wikidata.org/wiki/Q970 Comoros                   sovereign archipelago island nation in the Indian Ocean>
<EntityElement: https://www.wikidata.org/wiki/Q3806 Tübingen                  town in central Baden-Württemberg, Germany        >
<EntityElement: https://www.wikidata.org/wiki/Q4072 Göppingen                 town in Göppingen District, Stuttgart, Baden-Württemberg, Germany>
<EntityElement: https://www.wikidata.org/wiki/Q7565 father                    male parent or individual progenitor of human offspring>
<EntityElement: https://www.wikidata.org/wiki/Q43999 James                     major religious figure in Christian tradition and one of the Twelve Apostles>
<EntityElement: https://www.wikidata.